# 0. 개요

- 모든 시간대를 대상으로 하는 것이 아닌, 예측해야하는 시간대의 target이 비슷한 데이터를 바탕으로
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)
- 모델을 제작

# 1. 라이브러리 로드 및 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit, KFold

import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
import time
import datetime
import os

from scipy.sparse import csr_matrix

from mypackage import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 400)

### <div style="color:red">파일명 체크!!!</div>
- all_df: 기초작업 수준. 
- all_df4: lag 데이터 엄청 많이 만들어 놓은 것.
- all_df5: 'id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd', 'sales', 'date', 'wm_yr_wk', 'weekday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI', 'is_event', 'day', 'week', 'sell_price', 'revenue', sales_rolling_mean_t7', 'rolling_mean_t7', 'rolling_std_t7', 'sales_rolling_mean_t28', 'rolling_mean_t28', 'rolling_std_t28', 'sales_rolling_mean_t56', 'rolling_mean_t56', 'rolling_std_t56', 'sales_rolling_mean_t112','rolling_mean_t112', 'rolling_std_t112', 'sales_rolling_mean_t168', 'rolling_mean_t168', 'rolling_std_t168', 'lag_t28', 'lag_t29', 'lag_t30', 'lag_t31', 'lag_t32', 'lag_t33', 'lag_t34', 'revenue_lag_t28', 'revenue_lag_t29', 'revenue_lag_t30','revenue_lag_t31', 'revenue_lag_t32', 'revenue_lag_t33', 'revenue_lag_t34'

In [ ]:
print('data loading')
with open('inputs/all_df.pickle', 'rb') as f:
    all_df = pickle.load(f)
print('data loaded')

In [3]:
all_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,target,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,snap_CA,snap_TX,snap_WI,is_event,day,week,sell_price,lag_t28,lag_t29,lag_t30,lag_t24,lag_t25,lag_t26,lag_t27,rolling_mean_t7,rolling_std_t7,rolling_mean_t30,rolling_std_t30,rolling_mean_t90,rolling_mean_t180,rolling_mean_t60,rolling_std_t60,lag_price_t1,price_change_t1,rolling_price_max_t365,price_change_t365,rolling_price_std_t7,rolling_price_std_t30
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,0,2011-01-29,11101,2,1,2011,30,4,0,0,0,0,29,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,0,0,2011-01-29,11101,2,1,2011,30,4,0,0,0,0,29,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,0,0,2011-01-29,11101,2,1,2011,30,4,0,0,0,0,29,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,0,0,2011-01-29,11101,2,1,2011,30,4,0,0,0,0,29,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,0,0,2011-01-29,11101,2,1,2011,30,4,0,0,0,0,29,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
all_df['rolling_price_max_t180'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())

all_df['price_change_t180'] = (all_df['rolling_price_max_t180'] - all_df['sell_price']) / (all_df['rolling_price_max_t180'])

del all_df['rolling_price_max_t180']

In [ ]:
all_df['rolling_price_max_t365'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())

all_df['price_diff_t365'] = all_df['rolling_price_max_t365'] - all_df['sell_price']

del all_df['rolling_price_max_t365']

In [ ]:
all_df['rolling_price_max_t180'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())

all_df['price_change_t180']a = (all_df['rolling_price_max_t180'] - all_df['sell_price']) / (all_df['rolling_price_max_t180'])

del all_df['rolling_price_max_t180']

In [ ]:
all_df['rolling_price_max_t60'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())

all_df['price_change_t60'] = (all_df['rolling_price_max_t60'] - all_df['sell_price']) / (all_df['rolling_price_max_t60'])

del all_df['rolling_price_max_t60']

In [ ]:
all_df['rolling_price_max_t28'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())

all_df['price_change_t28'] = (all_df['rolling_price_max_t28'] - all_df['sell_price']) / (all_df['rolling_price_max_t28'])

del all_df['rolling_price_max_t28']

In [ ]:
all_df['rolling_price_std_t56'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(56).std())
all_df['rolling_price_std_t112'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(112).std())
all_df['rolling_price_std_t168'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(168).std())


# 2. 데이터 탐색

여기서는 test셋과 비슷한 시간대를 찾아서, 이 데이터만을 바탕으로 모델을 만들것임.  

test: 2016-04-25 ~ 2016-05-22  17~20주차 (16주차나 21주차까지 늘려야될지도 생각해보자)

---
train: 2015-04-27 ~ 2015-05-24  
train: 2014-04-28 ~ 2014-05-25  / 2014-04-21 ~ 2014-05-18
train: 2013-04-27 ~ 2013-05-24  
train: 2012-04-27 ~ 2012-05-24 
train: 2011-04-27 ~ 2011-05-24 

In [ ]:
len(all_df)

In [ ]:
all_df.tail()

In [4]:
# c00 = all_df['week'] == 9
# c01 = all_df['week'] == 10
# c02 = all_df['week'] == 11
# c03 = all_df['week'] == 12

c1 = all_df['week'] == 13
c2 = all_df['week'] == 14
c3 = all_df['week'] == 15
c4 = all_df['week'] == 16

c5 = all_df['week'] == 17
c6 = all_df['week'] == 18
c7 = all_df['week'] == 19
c8 = all_df['week'] == 20
all_df17_20 = all_df[c1 | c2 | c3 | c4 | c5 | c6 | c7 | c8]

In [ ]:
len(all_df17_20)

In [5]:
item_id_gb_sell_price_mean = all_df17_20.groupby(['store_id', 'item_id', 'year', 'week'])['sell_price'].mean().rename('item_id_gb_sell_price_mean')
# item_id_gb_sell_price_max = all_df17_20.groupby(['store_id', 'item_id', 'year', 'week'])['sell_price'].max().rename('item_id_gb_sell_price_max')
# item_id_gb_sell_price_min = all_df17_20.groupby(['store_id', 'item_id', 'year', 'week'])['sell_price'].min().rename('item_id_gb_sell_price_min')
item_id_gb_sell_price_std = all_df17_20.groupby(['store_id', 'item_id', 'year', 'week'])['sell_price'].std().rename('item_id_gb_sell_price_std')

all_df17_20 = pd.merge(all_df17_20, item_id_gb_sell_price_mean, on=['store_id', 'item_id', 'year', 'week'], how='left')
# all_df17_20 = pd.merge(all_df17_20, item_id_gb_sell_price_max, on=['store_id', 'item_id', 'year', 'week'], how='left')
# all_df17_20 = pd.merge(all_df17_20, item_id_gb_sell_price_min, on=['store_id', 'item_id', 'year', 'week'], how='left')
all_df17_20 = pd.merge(all_df17_20, item_id_gb_sell_price_std, on=['store_id', 'item_id', 'year', 'week'], how='left')

# item_id_gb_lag28 = all_df17_20.groupby(['store_id', 'item_id', 'year', 'week'])['lag_t28'].mean().rename('item_id_gb_lag28')
# all_df17_20 = pd.merge(all_df17_20, item_id_gb_lag28, on=['store_id', 'item_id', 'year', 'week'], how='left')

# all_df17_20['diff'] = all_df17_20['sell_price'] - all_df17_20['item_id_gb_sell_price']

In [ ]:
del all_df17_20['item_id_gb_sell_price_std']

In [ ]:
import pprint

In [ ]:
all_df17_20.groupby(['item_id', 'month'])['sell_price'].mean()

In [ ]:
# tmp = all_df17_20.groupby(['store_id', 'year','day'])['sales'].mean()

# for i in tmp.index:
#     print(i, tmp[i])

In [ ]:
all_df17_20.head()

In [ ]:
for i in item_id_gb_sell_price_mean:
    print(i)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10, 7))
sns.distplot(all_df17_20['event_name_1'])

In [6]:
from sklearn.preprocessing import LabelEncoder

le_id = LabelEncoder()
le_id.fit(all_df17_20['id'])

all_df17_20['id'] = le_id.transform(all_df17_20['id'])

# 3. 피쳐 엔지니어링

## 3.1 train, test

prepare training and test data.
- 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)

# 4. 모델 제작

In [ ]:
# from sklearn.ensemble import RandomForestRegressor

# rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)
# rf.fit(train2, np.log(train['target'] + 1))
# result = rf.predict(test2)

In [ ]:
# from lightgbm import LGBMRegressor
# # lgb = LGBMRegressor(num_leaves=2000, colsample_bytree=0.6, subsample=0.6, n_estimators=600, learning_rate=0.025, n_jobs=-1, device='gpu', max_bin = 63)
# lgb = LGBMRegressor(num_leaves=20, colsample_bytree=0.6, subsample=0.6, n_estimators=60, learning_rate=0.02, n_jobs=-1, device='cpu')

# lgb.fit(train, target)
# result = lgb.predict(test)

## 4.1 train set /  test set 선정

feature 선정에 있어서 다른 노트북에서는 all_df = all_df.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1) 이렇게 진행했음

In [11]:
all_df17_20.columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'month', 'year',
       'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI',
       'is_event', 'day', 'week', 'sell_price', 'lag_t28', 'lag_t29',
       'lag_t30', 'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30',
       'rolling_std_t30', 'rolling_mean_t90', 'rolling_mean_t180',
       'rolling_mean_t60', 'rolling_std_t60', 'lag_price_t1',
       'price_change_t1', 'rolling_price_max_t365', 'price_change_t365',
       'rolling_price_std_t7', 'rolling_price_std_t30',
       'item_id_gb_sell_price_mean', 'item_id_gb_sell_price_std']

In [12]:
all_df17_20.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,snap_CA,snap_TX,snap_WI,is_event,day,week,sell_price,lag_t28,lag_t29,lag_t30,lag_t24,lag_t25,lag_t26,lag_t27,rolling_mean_t7,rolling_std_t7,rolling_mean_t30,rolling_std_t30,rolling_mean_t90,rolling_mean_t180,rolling_mean_t60,rolling_std_t60,lag_price_t1,price_change_t1,rolling_price_max_t365,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,item_id_gb_sell_price_mean,item_id_gb_sell_price_std
0,14370,1437,3,1,0,0,1486,0,2011-03-28,11109,1,3,2011,30,4,0,0,0,0,28,13,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
drop_cols = ['d', 'sales', 'date', 'wm_yr_wk', 'is_event' ,'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27', 'lag_price_t1', 'rolling_price_max_t365']

features = all_df17_20.columns.drop(drop_cols)

In [18]:
features

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday',
       'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX',
       'snap_WI', 'day', 'week', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30',
       'rolling_std_t30', 'rolling_mean_t90', 'rolling_mean_t180',
       'rolling_mean_t60', 'rolling_std_t60', 'price_change_t1',
       'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30',
       'item_id_gb_sell_price_mean', 'item_id_gb_sell_price_std'],
      dtype='object')

In [19]:
train_set = all_df17_20[all_df17_20['date'] <= '2016-04-24']
train_set_X = train_set[features]
train_set_y = train_set['sales']

# 테스트 셋
test = all_df17_20[all_df17_20['date'] > '2016-04-24']
test_set = test[features]


var_set = all_df17_20[(all_df17_20['date'] > '2015-04-27') & (all_df17_20['date'] <= '2015-05-22')]
var_set_X = var_set[features]
var_set_y = var_set['sales']

In [ ]:
def split_store(data, store_id):
    data = data[data["store_id"] == store_id]
    return data

In [ ]:
def make_train_test_set(data, drop_cols):

    features = data.columns.drop(drop_cols)

    train_set = data[data['date'] <= '2016-04-24']
    train_set_X = train_set[features]
    train_set_y = train_set['target']

    # 테스트 셋
    test = data[data['date'] > '2016-04-24']
    test_set = test[features]

    var_set = data[(data['date'] > '2015-04-27') & (data['date'] <= '2015-05-22')]
    var_set_X = var_set[features]
    var_set_y = var_set['target']
    
    return train_set_X, train_set_y, test, test_set, var_set_X, var_set_y
    
    

In [ ]:
drop_cols = ['id', 'd', 'target', 'date', 'wm_yr_wk', 'is_event', 'lag_t28', 'lag_t29',
       'lag_t30', 'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27']

train_set_X, train_set_y, test, test_set, var_set_X, var_set_y = make_train_test_set(all_df17_20, drop_cols)

In [ ]:
all_df17_20[all_df17_20.store_id == 8]

## 4.2 KFold - LGBM 모델

In [ ]:
def lgb_model(data, drop_cols):
    train_set_X, train_set_y, test, test_set, var_set_X, var_set_y = make_train_test_set(data, drop_cols)
    n_fold = 5
    folds = KFold(n_splits=n_fold, shuffle=True, random_state=333)
    splits = folds.split(train_set_X, train_set_y)

    y_preds = np.zeros(test_set.shape[0])
    y_oof = np.zeros(train_set_X.shape[0])

    feature_importances = pd.DataFrame()
    feature_importances['feature'] = train_set_X.columns
    mean_score = []


    for fold_n, (train_index, valid_index) in enumerate(splits):
        print('Fold:',fold_n+1)

        X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
        y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]

        lgb = LGBMRegressor(
            boosting_type = 'gbdt',
            num_leaves = 2048,
            colsample_bytree = 0.8,
            subsample = 0.8,
            n_estimators = 300, ## 중요!!!!
            learning_rate = 0.1,
            n_jobs = -1,
            device = 'gpu',
            random_state=333
        )
        lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

        feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_

        y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)

        y_oof[valid_index] = y_pred_valid

        # validation 측정
        val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
        print(f'val rmse score is {val_score}')
        mean_score.append(val_score)

        all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
        all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
        print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')


        # 예측
        y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold

        # 메모리 정리
        del X_train, X_valid, y_train, y_valid

    features = train_set_X.columns
    params = lgb.get_params()
    eval_results = lgb.evals_result_['valid_0']['l2']

    print('mean rmse score over folds is',np.mean(mean_score))
    test['sales'] = y_preds
    write_params_features(features, params, eval_results)
    save_feature_importance(feature_importances)
    
    return test

In [20]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        objective = 'regression',
        boosting_type = 'gbdt',
        num_leaves = 2048,
        colsample_bytree = 0.8,
        subsample = 0.8,
        n_estimators = 800, ## 중요!!!!
        learning_rate = 0.05,
        n_jobs = -1,
        reg_lambda = 0.1,
        device = 'gpu'
    )
    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    mean_score.append(val_score)
    
    all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
    all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
    print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')
    
    
    # 예측
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = lgb.get_params()
eval_results = lgb.evals_result_['valid_0']['l2']

print('mean rmse score over folds is',np.mean(mean_score))
test['sales'] = y_preds



Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday',
       'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX',
       'snap_WI', 'day', 'week', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30',
       'rolling_std_t30', 'rolling_mean_t90', 'rolling_mean_t180',
       'rolling_mean_t60', 'rolling_std_t60', 'price_change_t1',
       'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30',
       'item_id_gb_sell_price_mean', 'item_id_gb_sell_price_std'],
      dtype='object')
Fold: 1
[1]	valid_0's l2: 13.0961
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 12.3295
[3]	valid_0's l2: 11.6412
[4]	valid_0's l2: 11.0074
[5]	valid_0's l2: 10.4331
[6]	valid_0's l2: 9.9168
[7]	valid_0's l2: 9.45158
[8]	valid_0's l2: 9.02575
[9]	valid_0's l2: 8.63981
[10]	valid_0's l2: 8.28794
[11]	valid_0's l2: 7.96763
[12]	valid_0's l2: 7.67612
[13]	valid_0's l2

[275]	valid_0's l2: 4.17543
[276]	valid_0's l2: 4.17559
[277]	valid_0's l2: 4.1752
[278]	valid_0's l2: 4.17422
[279]	valid_0's l2: 4.17448
[280]	valid_0's l2: 4.17466
[281]	valid_0's l2: 4.17476
[282]	valid_0's l2: 4.1736
[283]	valid_0's l2: 4.17325
[284]	valid_0's l2: 4.17256
[285]	valid_0's l2: 4.17258
[286]	valid_0's l2: 4.17256
[287]	valid_0's l2: 4.1721
[288]	valid_0's l2: 4.17195
[289]	valid_0's l2: 4.17175
[290]	valid_0's l2: 4.17122
[291]	valid_0's l2: 4.17065
[292]	valid_0's l2: 4.17052
[293]	valid_0's l2: 4.17075
[294]	valid_0's l2: 4.17026
[295]	valid_0's l2: 4.16977
[296]	valid_0's l2: 4.16909
[297]	valid_0's l2: 4.16738
[298]	valid_0's l2: 4.16715
[299]	valid_0's l2: 4.16723
[300]	valid_0's l2: 4.16676
[301]	valid_0's l2: 4.16679
[302]	valid_0's l2: 4.16641
[303]	valid_0's l2: 4.16634
[304]	valid_0's l2: 4.16546
[305]	valid_0's l2: 4.16505
[306]	valid_0's l2: 4.1649
[307]	valid_0's l2: 4.16435
[308]	valid_0's l2: 4.1643
[309]	valid_0's l2: 4.16402
[310]	valid_0's l2: 4.164

[569]	valid_0's l2: 4.11186
[570]	valid_0's l2: 4.11184
[571]	valid_0's l2: 4.11164
[572]	valid_0's l2: 4.11175
[573]	valid_0's l2: 4.11156
[574]	valid_0's l2: 4.11108
[575]	valid_0's l2: 4.11072
[576]	valid_0's l2: 4.11064
[577]	valid_0's l2: 4.11053
[578]	valid_0's l2: 4.11011
[579]	valid_0's l2: 4.10999
[580]	valid_0's l2: 4.11014
[581]	valid_0's l2: 4.10998
[582]	valid_0's l2: 4.10999
[583]	valid_0's l2: 4.10986
[584]	valid_0's l2: 4.10969
[585]	valid_0's l2: 4.10933
[586]	valid_0's l2: 4.10911
[587]	valid_0's l2: 4.10876
[588]	valid_0's l2: 4.10857
[589]	valid_0's l2: 4.10848
[590]	valid_0's l2: 4.10846
[591]	valid_0's l2: 4.10862
[592]	valid_0's l2: 4.10863
[593]	valid_0's l2: 4.10868
[594]	valid_0's l2: 4.10864
[595]	valid_0's l2: 4.10871
[596]	valid_0's l2: 4.1088
[597]	valid_0's l2: 4.10854
[598]	valid_0's l2: 4.1085
[599]	valid_0's l2: 4.10833
[600]	valid_0's l2: 4.10831
[601]	valid_0's l2: 4.10806
[602]	valid_0's l2: 4.10827
[603]	valid_0's l2: 4.10827
[604]	valid_0's l2: 4.

[56]	valid_0's l2: 4.36866
[57]	valid_0's l2: 4.3553
[58]	valid_0's l2: 4.3488
[59]	valid_0's l2: 4.3409
[60]	valid_0's l2: 4.33212
[61]	valid_0's l2: 4.3226
[62]	valid_0's l2: 4.31465
[63]	valid_0's l2: 4.30877
[64]	valid_0's l2: 4.30244
[65]	valid_0's l2: 4.29789
[66]	valid_0's l2: 4.28848
[67]	valid_0's l2: 4.27965
[68]	valid_0's l2: 4.27516
[69]	valid_0's l2: 4.27088
[70]	valid_0's l2: 4.26609
[71]	valid_0's l2: 4.2615
[72]	valid_0's l2: 4.25066
[73]	valid_0's l2: 4.24403
[74]	valid_0's l2: 4.24001
[75]	valid_0's l2: 4.23664
[76]	valid_0's l2: 4.23197
[77]	valid_0's l2: 4.22414
[78]	valid_0's l2: 4.22129
[79]	valid_0's l2: 4.21833
[80]	valid_0's l2: 4.21149
[81]	valid_0's l2: 4.20985
[82]	valid_0's l2: 4.20229
[83]	valid_0's l2: 4.19928
[84]	valid_0's l2: 4.19636
[85]	valid_0's l2: 4.18829
[86]	valid_0's l2: 4.17983
[87]	valid_0's l2: 4.17719
[88]	valid_0's l2: 4.17033
[89]	valid_0's l2: 4.16569
[90]	valid_0's l2: 4.15973
[91]	valid_0's l2: 4.15591
[92]	valid_0's l2: 4.15133
[93]	v

[352]	valid_0's l2: 3.83785
[353]	valid_0's l2: 3.83754
[354]	valid_0's l2: 3.83598
[355]	valid_0's l2: 3.83555
[356]	valid_0's l2: 3.8354
[357]	valid_0's l2: 3.83534
[358]	valid_0's l2: 3.83542
[359]	valid_0's l2: 3.83528
[360]	valid_0's l2: 3.8349
[361]	valid_0's l2: 3.83464
[362]	valid_0's l2: 3.83414
[363]	valid_0's l2: 3.83405
[364]	valid_0's l2: 3.83403
[365]	valid_0's l2: 3.83395
[366]	valid_0's l2: 3.83363
[367]	valid_0's l2: 3.83349
[368]	valid_0's l2: 3.83319
[369]	valid_0's l2: 3.833
[370]	valid_0's l2: 3.83304
[371]	valid_0's l2: 3.83293
[372]	valid_0's l2: 3.83187
[373]	valid_0's l2: 3.8321
[374]	valid_0's l2: 3.83165
[375]	valid_0's l2: 3.83164
[376]	valid_0's l2: 3.8313
[377]	valid_0's l2: 3.83132
[378]	valid_0's l2: 3.83109
[379]	valid_0's l2: 3.83029
[380]	valid_0's l2: 3.82994
[381]	valid_0's l2: 3.82967
[382]	valid_0's l2: 3.82986
[383]	valid_0's l2: 3.82928
[384]	valid_0's l2: 3.82947
[385]	valid_0's l2: 3.82912
[386]	valid_0's l2: 3.82934
[387]	valid_0's l2: 3.8291

...

[546]	valid_0's l2: 3.89668
[547]	valid_0's l2: 3.89666
[548]	valid_0's l2: 3.8961
[549]	valid_0's l2: 3.89583
[550]	valid_0's l2: 3.89566
[551]	valid_0's l2: 3.89563
[552]	valid_0's l2: 3.89503
[553]	valid_0's l2: 3.89504
[554]	valid_0's l2: 3.89521
[555]	valid_0's l2: 3.89519
[556]	valid_0's l2: 3.89509
[557]	valid_0's l2: 3.89508
[558]	valid_0's l2: 3.89472
[559]	valid_0's l2: 3.89474
[560]	valid_0's l2: 3.89463
[561]	valid_0's l2: 3.89458
[562]	valid_0's l2: 3.89457
[563]	valid_0's l2: 3.8944
[564]	valid_0's l2: 3.89426
[565]	valid_0's l2: 3.89386
[566]	valid_0's l2: 3.8939
[567]	valid_0's l2: 3.89384
[568]	valid_0's l2: 3.8936
[569]	valid_0's l2: 3.89361
[570]	valid_0's l2: 3.89352
[571]	valid_0's l2: 3.89288
[572]	valid_0's l2: 3.89273
[573]	valid_0's l2: 3.89272
[574]	valid_0's l2: 3.89256
[575]	valid_0's l2: 3.8925
[576]	valid_0's l2: 3.89237
[577]	valid_0's l2: 3.89235
[578]	valid_0's l2: 3.8923
[579]	valid_0's l2: 3.89243
[580]	valid_0's l2: 3.89208
[581]	valid_0's l2: 3.8920

[33]	valid_0's l2: 4.89941
[34]	valid_0's l2: 4.86268
[35]	valid_0's l2: 4.81897
[36]	valid_0's l2: 4.79078
[37]	valid_0's l2: 4.7586
[38]	valid_0's l2: 4.72957
[39]	valid_0's l2: 4.70681
[40]	valid_0's l2: 4.68579
[41]	valid_0's l2: 4.66421
[42]	valid_0's l2: 4.6451
[43]	valid_0's l2: 4.61869
[44]	valid_0's l2: 4.6038
[45]	valid_0's l2: 4.58602
[46]	valid_0's l2: 4.56975
[47]	valid_0's l2: 4.55407
[48]	valid_0's l2: 4.54181
[49]	valid_0's l2: 4.52902
[50]	valid_0's l2: 4.51498
[51]	valid_0's l2: 4.50442
[52]	valid_0's l2: 4.49069
[53]	valid_0's l2: 4.47618
[54]	valid_0's l2: 4.46567
[55]	valid_0's l2: 4.45512
[56]	valid_0's l2: 4.44723
[57]	valid_0's l2: 4.43178
[58]	valid_0's l2: 4.42487
[59]	valid_0's l2: 4.41778
[60]	valid_0's l2: 4.40746
[61]	valid_0's l2: 4.39701
[62]	valid_0's l2: 4.38688
[63]	valid_0's l2: 4.38123
[64]	valid_0's l2: 4.3738
[65]	valid_0's l2: 4.36662
[66]	valid_0's l2: 4.35627
[67]	valid_0's l2: 4.34678
[68]	valid_0's l2: 4.34232
[69]	valid_0's l2: 4.33754
[70]	

[330]	valid_0's l2: 3.90078
[331]	valid_0's l2: 3.90059
[332]	valid_0's l2: 3.90058
[333]	valid_0's l2: 3.8998
[334]	valid_0's l2: 3.89978
[335]	valid_0's l2: 3.89823
[336]	valid_0's l2: 3.89735
[337]	valid_0's l2: 3.89698
[338]	valid_0's l2: 3.89627
[339]	valid_0's l2: 3.89618
[340]	valid_0's l2: 3.89585
[341]	valid_0's l2: 3.89562
[342]	valid_0's l2: 3.89483
[343]	valid_0's l2: 3.89468
[344]	valid_0's l2: 3.89436
[345]	valid_0's l2: 3.89433
[346]	valid_0's l2: 3.89409
[347]	valid_0's l2: 3.89436
[348]	valid_0's l2: 3.89411
[349]	valid_0's l2: 3.89331
[350]	valid_0's l2: 3.89182
[351]	valid_0's l2: 3.89122
[352]	valid_0's l2: 3.89116
[353]	valid_0's l2: 3.89105
[354]	valid_0's l2: 3.89115
[355]	valid_0's l2: 3.89062
[356]	valid_0's l2: 3.89037
[357]	valid_0's l2: 3.89049
[358]	valid_0's l2: 3.89003
[359]	valid_0's l2: 3.88984
[360]	valid_0's l2: 3.88986
[361]	valid_0's l2: 3.88917
[362]	valid_0's l2: 3.88916
[363]	valid_0's l2: 3.88905
[364]	valid_0's l2: 3.88864
[365]	valid_0's l2: 3

[624]	valid_0's l2: 3.84096
[625]	valid_0's l2: 3.84092
[626]	valid_0's l2: 3.8408
[627]	valid_0's l2: 3.84062
[628]	valid_0's l2: 3.84043
[629]	valid_0's l2: 3.84048
[630]	valid_0's l2: 3.84036
[631]	valid_0's l2: 3.84022
[632]	valid_0's l2: 3.84015
[633]	valid_0's l2: 3.84005
[634]	valid_0's l2: 3.84004
[635]	valid_0's l2: 3.84004
[636]	valid_0's l2: 3.83985
[637]	valid_0's l2: 3.83962
[638]	valid_0's l2: 3.83948
[639]	valid_0's l2: 3.83921
[640]	valid_0's l2: 3.83932
[641]	valid_0's l2: 3.83933
[642]	valid_0's l2: 3.83917
[643]	valid_0's l2: 3.83918
[644]	valid_0's l2: 3.83906
[645]	valid_0's l2: 3.83884
[646]	valid_0's l2: 3.8388
[647]	valid_0's l2: 3.8387
[648]	valid_0's l2: 3.83866
[649]	valid_0's l2: 3.8387
[650]	valid_0's l2: 3.83865
[651]	valid_0's l2: 3.83821
[652]	valid_0's l2: 3.83828
[653]	valid_0's l2: 3.83768
[654]	valid_0's l2: 3.83746
[655]	valid_0's l2: 3.83754
[656]	valid_0's l2: 3.83763
[657]	valid_0's l2: 3.83761
[658]	valid_0's l2: 3.83767
[659]	valid_0's l2: 3.83

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
feature_importances

,feature,fold_1,fold_2,fold_3,fold_4,fold_5
0,id,71494,67157,72764,70951,72475
1,item_id,62996,60353,65217,65778,65621
2,dept_id,2939,2952,2795,2839,2953
3,cat_id,793,688,869,788,779
4,store_id,58016,53598,57276,56878,57667
5,state_id,10472,9680,10517,10400,10412
6,weekday,45387,42104,45596,45212,44353
7,month,8558,8370,8745,8697,8778
8,year,33362,30999,33493,32588,33422
9,event_name_1,3940,3611,3828,3790,3714


In [22]:
mean_score

[2.023259114084891,
 1.944939888740745,
 1.9570427104759294,
 1.9689079135651606,
 1.9569554379399332]

## 4.5 베이지안 최적화(Bayesian optimization
:  초매개변수(Hyper parameter)를 자동으로 조정

In [ ]:
from bayes_opt import BayesianOptimization
from functools import partial   

In [ ]:
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    kf = KFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
        model = LGBMRegressor(
            num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
            device = 'gpu'
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict(x_valid)
        true = y_valid
        score += metrics.mean_squared_error(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [ ]:
# 모델과 관련없는 변수 고정
func_fixed = partial(lgb_cv, x_data=train_set_X.reset_index(drop=True), y_data=train_set_y.reset_index(drop=True), n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (64, 4096),        # num_leaves,       범위(16~1024)
        'learning_rate': (0.0001, 0.1),  # learning_rate,    범위(0.0001~0.1)
        'n_estimators': (16, 1024),      # n_estimators,     범위(16~1024)
        'subsample': (0.4, 1),             # subsample,        범위(0~1)
        'colsample_bytree': (0.4, 1),      # colsample_bytree, 범위(0~1)
        'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),           # reg_lambda,       범위(0~50)
    }, 
    random_state=4321                    # 시드 고정
)
lgbBO.maximize(init_points=5, n_iter=30) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!

In [ ]:
feature_importances

## 4.3 store 별로 모델 만들기

In [ ]:
drop_cols = ['id', 'd', 'target', 'date', 'wm_yr_wk', 'is_event', 'lag_t28', 'lag_t29',
       'lag_t30', 'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27']

test_sets = []
for i in range(10):
    store_i = split_store(all_df17_20, i)
    test_sets.append(lgb_model(store_i, drop_cols))

In [ ]:
for i in range(1, 10):
    test_sets[0] = pd.concat([test_sets[0], test_sets[i]])

In [ ]:
test = test_sets[0].sort_index()

## 4.4 keras 모델

In [ ]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# model.compile(optimizer = "rmsprop", loss = root_mean_squared_error, 
#               metrics =["accuracy"])

In [ ]:
from keras.models import Sequential             # Sequential 생성자를 불러옵니다.
from keras.layers import Dense, Activation, Dropout      # Dense와 Activation 두 층 인스턴스를 불러옵니다.

model = Sequential()
model.add(Dense(128, activation='relu', kernel_initializer='he_normal', input_dim=train_set_X[features].shape[1]))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
# model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='linear'))                   # 1차원 출력에 'sigmoid' 함수를 적용하는 Dense 층입니다.
model.compile(optimizer='rmsprop',                          # 최적화 함수 = 'rmsprop'
              loss= 'mse',                   # 손실 함수 = 'binary_crossentropy'
              metrics=[root_mean_squared_error])                         # 평가 지표 = 'accuracy'

# 학습시키기 
model.fit(train_set_X, train_set_y, epochs=1, batch_size=1000, validation_split = 0.2)           # 생성된 데이터를 32개씩의 배치로 나누어 전체를 총 10회 학습시킵니다.

In [ ]:
test_set.head()

In [ ]:
result_ = model.predict(test_set)

In [ ]:
result_

In [ ]:
test['target'] = result_

# 5. 예측 및 제출

In [23]:
sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'sales']]
predictions['id'] = list(le_id.inverse_transform(predictions['id']))


predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])

for i in range(1,29):
    final['F'+str(i)] *= 1.0315
final.to_csv('submissions/submission.csv', index = False)

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [24]:
len(final)

60980

In [25]:
final.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.819884,0.706538,0.692868,0.691659,0.819518,1.010258,1.043801,0.756991,0.818024,0.777424,0.774483,0.990467,1.255375,1.071980,0.951746,0.813679,0.760173,0.796187,0.937410,1.354999,1.221252,0.865651,0.779631,0.779325,0.839429,0.918574,1.179945,1.220569
1,HOBBIES_1_002_CA_1_validation,0.390225,0.349317,0.337490,0.331118,0.380768,0.452096,0.458760,0.299616,0.277362,0.256142,0.245760,0.289219,0.362530,0.300807,0.246076,0.228317,0.225005,0.227535,0.268316,0.345341,0.352451,0.240076,0.219465,0.276946,0.263831,0.302730,0.379802,0.382578
2,HOBBIES_1_003_CA_1_validation,0.572337,0.506756,0.497018,0.508390,0.594450,0.660332,0.833394,0.503011,0.472170,0.449664,0.437171,0.507761,0.615086,0.519286,0.455379,0.415526,0.449022,0.490529,0.665553,0.797871,0.871757,0.692424,0.648511,0.634003,0.582080,0.660556,0.886086,0.869132
3,HOBBIES_1_004_CA_1_validation,1.900194,1.447776,1.391456,1.327046,1.890578,2.120231,2.469459,1.749918,1.637868,1.514949,1.460934,1.867600,2.571993,2.185294,2.053034,1.572096,1.709156,1.606156,1.868267,2.221166,2.500108,1.880574,1.567015,1.519098,1.536852,1.893421,2.700296,2.664174
4,HOBBIES_1_005_CA_1_validation,1.032826,0.874763,0.955769,1.056664,1.184156,1.529620,1.678152,1.171420,1.187487,1.026291,0.992747,1.168766,1.546845,1.192200,1.133994,0.964027,0.971829,0.989929,1.184118,1.556872,1.549171,0.999494,0.924931,0.922704,0.952170,1.126556,1.526201,1.687491


In [ ]:
# 새롭게 구한 값도 보정하니 정확도가 높아짐
# 1.04 를 곱하라고 했는데 1.03 곱하니 더 정확도가 높아짐
# 이차회귀식으로 계산 결과, 1.0315 곱하는게 가장 좋음.
# tmp = pd.read_csv('submissions/sub_dt_lgb.csv')
# for i in range(1,29):
#     tmp['F'+str(i)] *= 1.0315
# tmp.to_csv('submissions/submission.csv', index = False)

https://www.kaggle.com/c/m5-forecasting-accuracy/submit

In [26]:
time.sleep(2)
os.chdir("submissions")
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m lgb
os.chdir("../")

Successfully submitted to M5 Forecasting - Accuracy



  0%|          | 0.00/20.5M [00:00<?, ?B/s]
  0%|          | 8.00k/20.5M [00:00<10:15, 35.0kB/s]
  0%|          | 96.0k/20.5M [00:00<07:18, 48.9kB/s]
  2%|1         | 360k/20.5M [00:00<05:06, 69.2kB/s] 
  7%|6         | 1.42M/20.5M [00:00<03:23, 98.5kB/s]
 11%|#         | 2.23M/20.5M [00:00<02:17, 140kB/s] 
 13%|#3        | 2.67M/20.5M [00:01<01:41, 185kB/s]
 31%|###1      | 6.44M/20.5M [00:01<00:56, 263kB/s]
 38%|###7      | 7.79M/20.5M [00:01<00:35, 372kB/s]
 44%|####4     | 9.07M/20.5M [00:01<00:22, 525kB/s]
 50%|#####     | 10.3M/20.5M [00:01<00:14, 736kB/s]
 56%|#####6    | 11.5M/20.5M [00:01<00:09, 1.02MB/s]
 62%|######1   | 12.7M/20.5M [00:02<00:05, 1.41MB/s]
 67%|######7   | 13.9M/20.5M [00:02<00:03, 1.92MB/s]
 73%|#######2  | 15.0M/20.5M [00:02<00:02, 2.56MB/s]
 79%|#######8  | 16.1M/20.5M [00:02<00:01, 3.34MB/s]
 84%|########4 | 17.3M/20.5M [00:02<00:00, 4.24MB/s]
 90%|########9 | 18.4M/20.5M [00:02<00:00, 5.24MB/s]
 95%|#########5| 19.5M/20.5M [00:02<00:00, 6.29MB/s]
100%|#

# 6. 모델 파라미터 및 피처 기록 및 모델 저장하기

In [27]:
write_params_features(features, params, eval_results, mean_score)

In [28]:
save_feature_importance(feature_importances)

C:\Users\KJB\data_analysis\10_estimate_sales_of_walmart_retail_goods\mypackage\utils.py:134: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  fi = pd.concat([fi, tmp])
C:\Users\KJB\data_analysis\10_estimate_sales_of_walmart_retail_goods\mypackage\utils.py:135: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  fi = pd.concat([fi, feature])


In [ ]:
## 1등 노트북이랑 비교해서 카테고리형 피처 빠진거있나 확인하기

In [29]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        objective = 'regression',
        boosting_type = 'gbdt',
        num_leaves = 2048,
        colsample_bytree = 0.8,
        subsample = 0.8,
        n_estimators = 2000, ## 중요!!!!
        learning_rate = 0.025,
        n_jobs = -1,
        reg_lambda = 0.1,
        device = 'gpu'
    )
    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    mean_score.append(val_score)
    
    all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
    all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
    print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')
    
    
    # 예측
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = lgb.get_params()
eval_results = lgb.evals_result_['valid_0']['l2']

print('mean rmse score over folds is',np.mean(mean_score))
test['sales'] = y_preds



sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'sales']]
predictions['id'] = list(le_id.inverse_transform(predictions['id']))


predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])

for i in range(1,29):
    final['F'+str(i)] *= 1.0315
final.to_csv('submissions/submission.csv', index = False)

time.sleep(2)
os.chdir("submissions")
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m lgb
os.chdir("../")



Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday',
       'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX',
       'snap_WI', 'day', 'week', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30',
       'rolling_std_t30', 'rolling_mean_t90', 'rolling_mean_t180',
       'rolling_mean_t60', 'rolling_std_t60', 'price_change_t1',
       'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30',
       'item_id_gb_sell_price_mean', 'item_id_gb_sell_price_std'],
      dtype='object')
Fold: 1
[1]	valid_0's l2: 13.0322
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 12.6392
[3]	valid_0's l2: 12.2626
[4]	valid_0's l2: 11.8992
[5]	valid_0's l2: 11.5566
[6]	valid_0's l2: 11.2308
[7]	valid_0's l2: 10.9232
[8]	valid_0's l2: 10.6296
[9]	valid_0's l2: 10.347
[10]	valid_0's l2: 10.0765
[11]	valid_0's l2: 9.82064
[12]	valid_0's l2: 9.57747
[13]	valid_0's l2

[275]	valid_0's l2: 4.05968
[276]	valid_0's l2: 4.05803
[277]	valid_0's l2: 4.05585
[278]	valid_0's l2: 4.05532
[279]	valid_0's l2: 4.0538
[280]	valid_0's l2: 4.05343
[281]	valid_0's l2: 4.05289
[282]	valid_0's l2: 4.05213
[283]	valid_0's l2: 4.04952
[284]	valid_0's l2: 4.04736
[285]	valid_0's l2: 4.04681
[286]	valid_0's l2: 4.04654
[287]	valid_0's l2: 4.04444
[288]	valid_0's l2: 4.04402
[289]	valid_0's l2: 4.04125
[290]	valid_0's l2: 4.04084
[291]	valid_0's l2: 4.04038
[292]	valid_0's l2: 4.03871
[293]	valid_0's l2: 4.03839
[294]	valid_0's l2: 4.03783
[295]	valid_0's l2: 4.0359
[296]	valid_0's l2: 4.03495
[297]	valid_0's l2: 4.03388
[298]	valid_0's l2: 4.03331
[299]	valid_0's l2: 4.0314
[300]	valid_0's l2: 4.03076
[301]	valid_0's l2: 4.03055
[302]	valid_0's l2: 4.02989
[303]	valid_0's l2: 4.02993
[304]	valid_0's l2: 4.02985
[305]	valid_0's l2: 4.02919
[306]	valid_0's l2: 4.02888
[307]	valid_0's l2: 4.02803
[308]	valid_0's l2: 4.0277
[309]	valid_0's l2: 4.02761
[310]	valid_0's l2: 4.02

[569]	valid_0's l2: 3.92373
[570]	valid_0's l2: 3.92371
[571]	valid_0's l2: 3.92291
[572]	valid_0's l2: 3.92264
[573]	valid_0's l2: 3.92269
[574]	valid_0's l2: 3.92255
[575]	valid_0's l2: 3.92227
[576]	valid_0's l2: 3.92205
[577]	valid_0's l2: 3.92174
[578]	valid_0's l2: 3.92165
[579]	valid_0's l2: 3.92167
[580]	valid_0's l2: 3.92145
[581]	valid_0's l2: 3.92139
[582]	valid_0's l2: 3.92103
[583]	valid_0's l2: 3.92092
[584]	valid_0's l2: 3.9206
[585]	valid_0's l2: 3.92041
[586]	valid_0's l2: 3.92025
[587]	valid_0's l2: 3.92011
[588]	valid_0's l2: 3.91991
[589]	valid_0's l2: 3.91962
[590]	valid_0's l2: 3.91944
[591]	valid_0's l2: 3.91944
[592]	valid_0's l2: 3.91938
[593]	valid_0's l2: 3.91911
[594]	valid_0's l2: 3.91897
[595]	valid_0's l2: 3.91902
[596]	valid_0's l2: 3.91894
[597]	valid_0's l2: 3.91842
[598]	valid_0's l2: 3.91817
[599]	valid_0's l2: 3.91825
[600]	valid_0's l2: 3.91819
[601]	valid_0's l2: 3.91764
[602]	valid_0's l2: 3.91714
[603]	valid_0's l2: 3.91697
[604]	valid_0's l2: 3

[863]	valid_0's l2: 3.88329
[864]	valid_0's l2: 3.88306
[865]	valid_0's l2: 3.88304
[866]	valid_0's l2: 3.88289
[867]	valid_0's l2: 3.88271
[868]	valid_0's l2: 3.88253
[869]	valid_0's l2: 3.88238
[870]	valid_0's l2: 3.88228
[871]	valid_0's l2: 3.88227
[872]	valid_0's l2: 3.88227
[873]	valid_0's l2: 3.88208
[874]	valid_0's l2: 3.88202
[875]	valid_0's l2: 3.8821
[876]	valid_0's l2: 3.8821
[877]	valid_0's l2: 3.8822
[878]	valid_0's l2: 3.88224
[879]	valid_0's l2: 3.88214
[880]	valid_0's l2: 3.88218
[881]	valid_0's l2: 3.88178
[882]	valid_0's l2: 3.88169
[883]	valid_0's l2: 3.88166
[884]	valid_0's l2: 3.88147
[885]	valid_0's l2: 3.88139
[886]	valid_0's l2: 3.88141
[887]	valid_0's l2: 3.88112
[888]	valid_0's l2: 3.88101
[889]	valid_0's l2: 3.88083
[890]	valid_0's l2: 3.88088
[891]	valid_0's l2: 3.88059
[892]	valid_0's l2: 3.88059
[893]	valid_0's l2: 3.88056
[894]	valid_0's l2: 3.88045
[895]	valid_0's l2: 3.88028
[896]	valid_0's l2: 3.88022
[897]	valid_0's l2: 3.88012
[898]	valid_0's l2: 3.8

[223]	valid_0's l2: 4.06516
[224]	valid_0's l2: 4.0631
[225]	valid_0's l2: 4.06118
[226]	valid_0's l2: 4.05797
[227]	valid_0's l2: 4.0567
[228]	valid_0's l2: 4.05484
[229]	valid_0's l2: 4.05243
[230]	valid_0's l2: 4.05104
[231]	valid_0's l2: 4.04994
[232]	valid_0's l2: 4.04609
[233]	valid_0's l2: 4.0452
[234]	valid_0's l2: 4.04396
[235]	valid_0's l2: 4.04033
[236]	valid_0's l2: 4.03801
[237]	valid_0's l2: 4.03645
[238]	valid_0's l2: 4.03419
[239]	valid_0's l2: 4.03229
[240]	valid_0's l2: 4.03064
[241]	valid_0's l2: 4.02716
[242]	valid_0's l2: 4.02599
[243]	valid_0's l2: 4.02505
[244]	valid_0's l2: 4.02396
[245]	valid_0's l2: 4.02304
[246]	valid_0's l2: 4.02195
[247]	valid_0's l2: 4.02119
[248]	valid_0's l2: 4.02054
[249]	valid_0's l2: 4.01906
[250]	valid_0's l2: 4.01851
[251]	valid_0's l2: 4.0175
[252]	valid_0's l2: 4.01682
[253]	valid_0's l2: 4.01575
[254]	valid_0's l2: 4.0129
[255]	valid_0's l2: 4.01247
[256]	valid_0's l2: 4.01191
[257]	valid_0's l2: 4.01082
[258]	valid_0's l2: 4.009

...

[1236]	valid_0's l2: 3.81138
[1237]	valid_0's l2: 3.81137
[1238]	valid_0's l2: 3.81141
[1239]	valid_0's l2: 3.81142
[1240]	valid_0's l2: 3.8114
[1241]	valid_0's l2: 3.81131
[1242]	valid_0's l2: 3.81122
[1243]	valid_0's l2: 3.81115
[1244]	valid_0's l2: 3.81116
[1245]	valid_0's l2: 3.81106
[1246]	valid_0's l2: 3.81099
[1247]	valid_0's l2: 3.81109
[1248]	valid_0's l2: 3.81105
[1249]	valid_0's l2: 3.81101
[1250]	valid_0's l2: 3.81101
[1251]	valid_0's l2: 3.81103
[1252]	valid_0's l2: 3.81102
[1253]	valid_0's l2: 3.81094
[1254]	valid_0's l2: 3.81079
[1255]	valid_0's l2: 3.81061
[1256]	valid_0's l2: 3.81039
[1257]	valid_0's l2: 3.81037
[1258]	valid_0's l2: 3.81041
[1259]	valid_0's l2: 3.81041
[1260]	valid_0's l2: 3.81049
[1261]	valid_0's l2: 3.81046
[1262]	valid_0's l2: 3.81045
[1263]	valid_0's l2: 3.81051
[1264]	valid_0's l2: 3.81043
[1265]	valid_0's l2: 3.81036
[1266]	valid_0's l2: 3.8104
[1267]	valid_0's l2: 3.81048
[1268]	valid_0's l2: 3.81051
[1269]	valid_0's l2: 3.81039
[1270]	valid_0's

[1520]	valid_0's l2: 3.79916
[1521]	valid_0's l2: 3.79918
[1522]	valid_0's l2: 3.79918
[1523]	valid_0's l2: 3.79898
[1524]	valid_0's l2: 3.79891
[1525]	valid_0's l2: 3.79887
[1526]	valid_0's l2: 3.79879
[1527]	valid_0's l2: 3.79878
[1528]	valid_0's l2: 3.79874
[1529]	valid_0's l2: 3.79872
[1530]	valid_0's l2: 3.79858
[1531]	valid_0's l2: 3.79854
[1532]	valid_0's l2: 3.79841
[1533]	valid_0's l2: 3.79833
[1534]	valid_0's l2: 3.79834
[1535]	valid_0's l2: 3.79836
[1536]	valid_0's l2: 3.7984
[1537]	valid_0's l2: 3.79834
[1538]	valid_0's l2: 3.79829
[1539]	valid_0's l2: 3.79825
[1540]	valid_0's l2: 3.79823
[1541]	valid_0's l2: 3.7982
[1542]	valid_0's l2: 3.79814
[1543]	valid_0's l2: 3.79812
[1544]	valid_0's l2: 3.79811
[1545]	valid_0's l2: 3.79813
[1546]	valid_0's l2: 3.79809
[1547]	valid_0's l2: 3.79787
[1548]	valid_0's l2: 3.79786
[1549]	valid_0's l2: 3.79785
[1550]	valid_0's l2: 3.79782
[1551]	valid_0's l2: 3.79775
[1552]	valid_0's l2: 3.79772
[1553]	valid_0's l2: 3.79771
[1554]	valid_0's

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Successfully submitted to M5 Forecasting - Accuracy



  0%|          | 0.00/20.5M [00:00<?, ?B/s]
  0%|          | 8.00k/20.5M [00:00<09:01, 39.8kB/s]
  0%|          | 96.0k/20.5M [00:00<06:25, 55.6kB/s]
  2%|1         | 360k/20.5M [00:00<04:29, 78.6kB/s] 

In [30]:
write_params_features(features, params, eval_results, mean_score)
save_feature_importance(feature_importances)


  7%|6         | 1.42M/20.5M [00:00<02:59, 112kB/s]
 11%|#1        | 2.28M/20.5M [00:00<02:00, 159kB/s]
 13%|#3        | 2.77M/20.5M [00:02<01:40, 186kB/s]
 32%|###1      | 6.50M/20.5M [00:02<00:55, 265kB/s]
 38%|###8      | 7.85M/20.5M [00:02<00:35, 375kB/s]
 44%|####4     | 9.13M/20.5M [00:02<00:22, 528kB/s]
 50%|#####     | 10.4M/20.5M [00:02<00:14, 740kB/s]
 56%|#####6    | 11.5M/20.5M [00:02<00:09, 1.03MB/s]
 62%|######1   | 12.7M/20.5M [00:02<00:05, 1.42MB/s]
 68%|######7   | 13.9M/20.5M [00:03<00:03, 1.93MB/s]
 73%|#######3  | 15.0M/20.5M [00:03<00:02, 2.57MB/s]
 79%|#######8  | 16.2M/20.5M [00:03<00:01, 3.36MB/s]
 84%|########4 | 17.3M/20.5M [00:03<00:00, 4.27MB/s]
 90%|########9 | 18.4M/20.5M [00:03<00:00, 5.28MB/s]
 95%|#########5| 19.6M/20.5M [00:03<00:00, 6.34MB/s]
100%|##########| 20.5M/20.5M [00:07<00:00, 2.75MB/s]
C:\Users\KJB\data_analysis\10_estimate_sales_of_walmart_retail_goods\mypackage\utils.py:134: FutureWarning: Sorting because non-concatenation axis is not alig

In [31]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        objective = 'regression',
        boosting_type = 'gbdt',
        num_leaves = 2048,
        colsample_bytree = 0.8,
        subsample = 0.8,
        n_estimators = 800, ## 중요!!!!
        learning_rate = 0.05,
        n_jobs = -1,
        reg_lambda = 10,
        device = 'gpu'
    )
    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    mean_score.append(val_score)
    
    all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
    all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
    print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')
    
    
    # 예측
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = lgb.get_params()
eval_results = lgb.evals_result_['valid_0']['l2']

print('mean rmse score over folds is',np.mean(mean_score))
test['sales'] = y_preds



sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'sales']]
predictions['id'] = list(le_id.inverse_transform(predictions['id']))


predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])

for i in range(1,29):
    final['F'+str(i)] *= 1.0315
final.to_csv('submissions/submission.csv', index = False)

time.sleep(2)
os.chdir("submissions")
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m lgb
os.chdir("../")

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday',
       'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX',
       'snap_WI', 'day', 'week', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30',
       'rolling_std_t30', 'rolling_mean_t90', 'rolling_mean_t180',
       'rolling_mean_t60', 'rolling_std_t60', 'price_change_t1',
       'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30',
       'item_id_gb_sell_price_mean', 'item_id_gb_sell_price_std'],
      dtype='object')
Fold: 1
[1]	valid_0's l2: 12.5453
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 11.8429
[3]	valid_0's l2: 11.2071
[4]	valid_0's l2: 10.6177
[5]	valid_0's l2: 10.0905
[6]	valid_0's l2: 9.61338
[7]	valid_0's l2: 9.18115
[8]	valid_0's l2: 8.78763
[9]	valid_0's l2: 8.42855
[10]	valid_0's l2: 8.09816
[11]	valid_0's l2: 7.80273
[12]	valid_0's l2: 7.53187
[13]	valid_0's l

[275]	valid_0's l2: 3.97384
[276]	valid_0's l2: 3.97358
[277]	valid_0's l2: 3.97329
[278]	valid_0's l2: 3.97322
[279]	valid_0's l2: 3.97332
[280]	valid_0's l2: 3.97304
[281]	valid_0's l2: 3.97158
[282]	valid_0's l2: 3.9709
[283]	valid_0's l2: 3.97059
[284]	valid_0's l2: 3.97055
[285]	valid_0's l2: 3.97028
[286]	valid_0's l2: 3.96983
[287]	valid_0's l2: 3.96898
[288]	valid_0's l2: 3.96842
[289]	valid_0's l2: 3.96778
[290]	valid_0's l2: 3.96699
[291]	valid_0's l2: 3.96638
[292]	valid_0's l2: 3.96606
[293]	valid_0's l2: 3.96608
[294]	valid_0's l2: 3.96597
[295]	valid_0's l2: 3.96559
[296]	valid_0's l2: 3.96441
[297]	valid_0's l2: 3.96407
[298]	valid_0's l2: 3.96354
[299]	valid_0's l2: 3.96295
[300]	valid_0's l2: 3.9626
[301]	valid_0's l2: 3.96216
[302]	valid_0's l2: 3.96096
[303]	valid_0's l2: 3.96029
[304]	valid_0's l2: 3.95982
[305]	valid_0's l2: 3.95963
[306]	valid_0's l2: 3.95906
[307]	valid_0's l2: 3.9584
[308]	valid_0's l2: 3.95837
[309]	valid_0's l2: 3.95845
[310]	valid_0's l2: 3.9

[569]	valid_0's l2: 3.90048
[570]	valid_0's l2: 3.90004
[571]	valid_0's l2: 3.90012
[572]	valid_0's l2: 3.89919
[573]	valid_0's l2: 3.8991
[574]	valid_0's l2: 3.89918
[575]	valid_0's l2: 3.89841
[576]	valid_0's l2: 3.89847
[577]	valid_0's l2: 3.89844
[578]	valid_0's l2: 3.89834
[579]	valid_0's l2: 3.89841
[580]	valid_0's l2: 3.89818
[581]	valid_0's l2: 3.89821
[582]	valid_0's l2: 3.89808
[583]	valid_0's l2: 3.8981
[584]	valid_0's l2: 3.89777
[585]	valid_0's l2: 3.8977
[586]	valid_0's l2: 3.89731
[587]	valid_0's l2: 3.89725
[588]	valid_0's l2: 3.89724
[589]	valid_0's l2: 3.89713
[590]	valid_0's l2: 3.89687
[591]	valid_0's l2: 3.89687
[592]	valid_0's l2: 3.8967
[593]	valid_0's l2: 3.89665
[594]	valid_0's l2: 3.89616
[595]	valid_0's l2: 3.8961
[596]	valid_0's l2: 3.8959
[597]	valid_0's l2: 3.89546
[598]	valid_0's l2: 3.89528
[599]	valid_0's l2: 3.89528
[600]	valid_0's l2: 3.89536
[601]	valid_0's l2: 3.89506
[602]	valid_0's l2: 3.89474
[603]	valid_0's l2: 3.89458
[604]	valid_0's l2: 3.8942

[105]	valid_0's l2: 4.23496
[106]	valid_0's l2: 4.23172
[107]	valid_0's l2: 4.2252
[108]	valid_0's l2: 4.22112
[109]	valid_0's l2: 4.21638
[110]	valid_0's l2: 4.21247
[111]	valid_0's l2: 4.21041
[112]	valid_0's l2: 4.20538
[113]	valid_0's l2: 4.20192
[114]	valid_0's l2: 4.19888
[115]	valid_0's l2: 4.1956
[116]	valid_0's l2: 4.19271
[117]	valid_0's l2: 4.18905
[118]	valid_0's l2: 4.1856
[119]	valid_0's l2: 4.18398
[120]	valid_0's l2: 4.18214
[121]	valid_0's l2: 4.18084
[122]	valid_0's l2: 4.17849
[123]	valid_0's l2: 4.17565
[124]	valid_0's l2: 4.17435
[125]	valid_0's l2: 4.17062
[126]	valid_0's l2: 4.16841
[127]	valid_0's l2: 4.16504
[128]	valid_0's l2: 4.16355
[129]	valid_0's l2: 4.16154
[130]	valid_0's l2: 4.15727
[131]	valid_0's l2: 4.15513
[132]	valid_0's l2: 4.15062
[133]	valid_0's l2: 4.14883
[134]	valid_0's l2: 4.14662
[135]	valid_0's l2: 4.14275
[136]	valid_0's l2: 4.13901
[137]	valid_0's l2: 4.13519
[138]	valid_0's l2: 4.13265
[139]	valid_0's l2: 4.12865
[140]	valid_0's l2: 4.1

[399]	valid_0's l2: 3.93367
[400]	valid_0's l2: 3.93372
[401]	valid_0's l2: 3.93359
[402]	valid_0's l2: 3.93372
[403]	valid_0's l2: 3.9336
[404]	valid_0's l2: 3.93387
[405]	valid_0's l2: 3.93416
[406]	valid_0's l2: 3.93387
[407]	valid_0's l2: 3.93395
[408]	valid_0's l2: 3.93365
[409]	valid_0's l2: 3.93351
[410]	valid_0's l2: 3.93328
[411]	valid_0's l2: 3.93349
[412]	valid_0's l2: 3.93335
[413]	valid_0's l2: 3.93353
[414]	valid_0's l2: 3.93346
[415]	valid_0's l2: 3.93344
[416]	valid_0's l2: 3.93344
[417]	valid_0's l2: 3.93347
[418]	valid_0's l2: 3.93339
[419]	valid_0's l2: 3.93339
[420]	valid_0's l2: 3.93314
[421]	valid_0's l2: 3.93297
[422]	valid_0's l2: 3.93276
[423]	valid_0's l2: 3.93272
[424]	valid_0's l2: 3.93264
[425]	valid_0's l2: 3.93252
[426]	valid_0's l2: 3.93241
[427]	valid_0's l2: 3.93261
[428]	valid_0's l2: 3.93274
[429]	valid_0's l2: 3.93274
[430]	valid_0's l2: 3.933
[431]	valid_0's l2: 3.93309
[432]	valid_0's l2: 3.933
[433]	valid_0's l2: 3.93292
[434]	valid_0's l2: 3.932

...

[211]	valid_0's l2: 3.93679
[212]	valid_0's l2: 3.93547
[213]	valid_0's l2: 3.93422
[214]	valid_0's l2: 3.93372
[215]	valid_0's l2: 3.93323
[216]	valid_0's l2: 3.93132
[217]	valid_0's l2: 3.93031
[218]	valid_0's l2: 3.92998
[219]	valid_0's l2: 3.92941
[220]	valid_0's l2: 3.92856
[221]	valid_0's l2: 3.92763
[222]	valid_0's l2: 3.92612
[223]	valid_0's l2: 3.92532
[224]	valid_0's l2: 3.92389
[225]	valid_0's l2: 3.92282
[226]	valid_0's l2: 3.92211
[227]	valid_0's l2: 3.92142
[228]	valid_0's l2: 3.91863
[229]	valid_0's l2: 3.91668
[230]	valid_0's l2: 3.91604
[231]	valid_0's l2: 3.91451
[232]	valid_0's l2: 3.9139
[233]	valid_0's l2: 3.91366
[234]	valid_0's l2: 3.91231
[235]	valid_0's l2: 3.91183
[236]	valid_0's l2: 3.911
[237]	valid_0's l2: 3.91023
[238]	valid_0's l2: 3.90888
[239]	valid_0's l2: 3.90655
[240]	valid_0's l2: 3.90526
[241]	valid_0's l2: 3.9038
[242]	valid_0's l2: 3.90358
[243]	valid_0's l2: 3.90347
[244]	valid_0's l2: 3.90317
[245]	valid_0's l2: 3.90319
[246]	valid_0's l2: 3.90

[505]	valid_0's l2: 3.82187
[506]	valid_0's l2: 3.82188
[507]	valid_0's l2: 3.82181
[508]	valid_0's l2: 3.82129
[509]	valid_0's l2: 3.8209
[510]	valid_0's l2: 3.82082
[511]	valid_0's l2: 3.82057
[512]	valid_0's l2: 3.82042
[513]	valid_0's l2: 3.81966
[514]	valid_0's l2: 3.81972
[515]	valid_0's l2: 3.81943
[516]	valid_0's l2: 3.81921
[517]	valid_0's l2: 3.81919
[518]	valid_0's l2: 3.81919
[519]	valid_0's l2: 3.819
[520]	valid_0's l2: 3.81822
[521]	valid_0's l2: 3.81832
[522]	valid_0's l2: 3.8181
[523]	valid_0's l2: 3.81802
[524]	valid_0's l2: 3.81808
[525]	valid_0's l2: 3.81792
[526]	valid_0's l2: 3.81787
[527]	valid_0's l2: 3.81786
[528]	valid_0's l2: 3.8179
[529]	valid_0's l2: 3.81802
[530]	valid_0's l2: 3.81809
[531]	valid_0's l2: 3.81777
[532]	valid_0's l2: 3.81774
[533]	valid_0's l2: 3.81784
[534]	valid_0's l2: 3.81788
[535]	valid_0's l2: 3.81796
[536]	valid_0's l2: 3.81797
[537]	valid_0's l2: 3.81794
[538]	valid_0's l2: 3.81797
[539]	valid_0's l2: 3.81767
[540]	valid_0's l2: 3.817

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Successfully submitted to M5 Forecasting - Accuracy



  0%|          | 0.00/20.5M [00:00<?, ?B/s]
  0%|          | 8.00k/20.5M [00:00<09:51, 36.4kB/s]
  0%|          | 96.0k/20.5M [00:00<07:01, 50.9kB/s]
  2%|1         | 360k/20.5M [00:00<04:54, 71.9kB/s] 
  7%|6         | 1.42M/20.5M [00:00<03:15, 102kB/s]
 11%|#         | 2.22M/20.5M [00:00<02:12, 146kB/s]
 13%|#3        | 2.67M/20.5M [00:02<01:50, 169kB/s]
 31%|###1      | 6.43M/20.5M [00:02<01:01, 241kB/s]
 38%|###7      | 7.78M/20.5M [00:02<00:39, 341kB/s]
 44%|####4     | 9.06M/20.5M [00:02<00:25, 481kB/s]
 50%|#####     | 10.3M/20.5M [00:02<00:15, 676kB/s]
 56%|#####5    | 11.5M/20.5M [00:03<00:10, 942kB/s]
 62%|######1   | 12.7M/20.5M [00:03<00:06, 1.30MB/s]
 67%|######7   | 13.8M/20.5M [00:03<00:03, 1.77MB/s]
 73%|#######2  | 15.0M/20.5M [00:03<00:02, 2.38MB/s]
 78%|#######8  | 16.1M/20.5M [00:03<00:01, 3.13MB/s]
 84%|########4 | 17.3M/20.5M [00:03<00:00, 4.01MB/s]
 90%|########9 | 18.4M/20.5M [00:03<00:00, 5.00MB/s]
 95%|#########5| 19.5M/20.5M [00:03<00:00, 6.04MB/s]
100%|####

In [32]:
write_params_features(features, params, eval_results, mean_score)
save_feature_importance(feature_importances)

C:\Users\KJB\data_analysis\10_estimate_sales_of_walmart_retail_goods\mypackage\utils.py:134: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  fi = pd.concat([fi, tmp])
C:\Users\KJB\data_analysis\10_estimate_sales_of_walmart_retail_goods\mypackage\utils.py:135: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  fi = pd.concat([fi, feature])


In [33]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        objective = 'regression',
        boosting_type = 'gbdt',
        num_leaves = 2048,
        colsample_bytree = 0.8,
        subsample = 0.8,
        n_estimators = 800, ## 중요!!!!
        learning_rate = 0.05,
        n_jobs = -1,
        reg_lambda = 100,
        device = 'gpu'
    )
    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    mean_score.append(val_score)
    
    all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
    all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
    print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')
    
    
    # 예측
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = lgb.get_params()
eval_results = lgb.evals_result_['valid_0']['l2']

print('mean rmse score over folds is',np.mean(mean_score))
test['sales'] = y_preds



sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'sales']]
predictions['id'] = list(le_id.inverse_transform(predictions['id']))


predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])

for i in range(1,29):
    final['F'+str(i)] *= 1.0315
final.to_csv('submissions/submission.csv', index = False)

time.sleep(2)
os.chdir("submissions")
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m lgb
os.chdir("../")

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday',
       'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX',
       'snap_WI', 'day', 'week', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30',
       'rolling_std_t30', 'rolling_mean_t90', 'rolling_mean_t180',
       'rolling_mean_t60', 'rolling_std_t60', 'price_change_t1',
       'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30',
       'item_id_gb_sell_price_mean', 'item_id_gb_sell_price_std'],
      dtype='object')
Fold: 1
[1]	valid_0's l2: 12.868
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 12.1814
[3]	valid_0's l2: 11.5558
[4]	valid_0's l2: 10.9859
[5]	valid_0's l2: 10.4705
[6]	valid_0's l2: 9.99927
[7]	valid_0's l2: 9.57208
[8]	valid_0's l2: 9.18323
[9]	valid_0's l2: 8.83089
[10]	valid_0's l2: 8.50319
[11]	valid_0's l2: 8.20902
[12]	valid_0's l2: 7.93974
[13]	valid_0's l2

[275]	valid_0's l2: 4.0656
[276]	valid_0's l2: 4.06479
[277]	valid_0's l2: 4.06441
[278]	valid_0's l2: 4.06326
[279]	valid_0's l2: 4.06282
[280]	valid_0's l2: 4.06256
[281]	valid_0's l2: 4.06204
[282]	valid_0's l2: 4.06137
[283]	valid_0's l2: 4.05991
[284]	valid_0's l2: 4.05952
[285]	valid_0's l2: 4.05908
[286]	valid_0's l2: 4.05879
[287]	valid_0's l2: 4.05851
[288]	valid_0's l2: 4.05823
[289]	valid_0's l2: 4.05796
[290]	valid_0's l2: 4.05749
[291]	valid_0's l2: 4.05741
[292]	valid_0's l2: 4.05677
[293]	valid_0's l2: 4.05652
[294]	valid_0's l2: 4.05616
[295]	valid_0's l2: 4.05592
[296]	valid_0's l2: 4.05438
[297]	valid_0's l2: 4.05398
[298]	valid_0's l2: 4.05367
[299]	valid_0's l2: 4.0527
[300]	valid_0's l2: 4.05252
[301]	valid_0's l2: 4.05243
[302]	valid_0's l2: 4.05162
[303]	valid_0's l2: 4.05127
[304]	valid_0's l2: 4.05033
[305]	valid_0's l2: 4.04942
[306]	valid_0's l2: 4.04914
[307]	valid_0's l2: 4.04858
[308]	valid_0's l2: 4.04816
[309]	valid_0's l2: 4.04774
[310]	valid_0's l2: 4.

[569]	valid_0's l2: 3.96019
[570]	valid_0's l2: 3.95927
[571]	valid_0's l2: 3.95903
[572]	valid_0's l2: 3.95888
[573]	valid_0's l2: 3.95865
[574]	valid_0's l2: 3.95839
[575]	valid_0's l2: 3.95818
[576]	valid_0's l2: 3.95811
[577]	valid_0's l2: 3.95794
[578]	valid_0's l2: 3.95788
[579]	valid_0's l2: 3.95797
[580]	valid_0's l2: 3.95796
[581]	valid_0's l2: 3.95786
[582]	valid_0's l2: 3.95744
[583]	valid_0's l2: 3.95723
[584]	valid_0's l2: 3.95672
[585]	valid_0's l2: 3.95654
[586]	valid_0's l2: 3.95641
[587]	valid_0's l2: 3.95631
[588]	valid_0's l2: 3.95619
[589]	valid_0's l2: 3.95616
[590]	valid_0's l2: 3.95609
[591]	valid_0's l2: 3.95613
[592]	valid_0's l2: 3.95621
[593]	valid_0's l2: 3.95615
[594]	valid_0's l2: 3.95605
[595]	valid_0's l2: 3.95601
[596]	valid_0's l2: 3.956
[597]	valid_0's l2: 3.95594
[598]	valid_0's l2: 3.95592
[599]	valid_0's l2: 3.95585
[600]	valid_0's l2: 3.95566
[601]	valid_0's l2: 3.95556
[602]	valid_0's l2: 3.95558
[603]	valid_0's l2: 3.9555
[604]	valid_0's l2: 3.9

[56]	valid_0's l2: 4.75844
[57]	valid_0's l2: 4.74023
[58]	valid_0's l2: 4.72651
[59]	valid_0's l2: 4.71095
[60]	valid_0's l2: 4.69675
[61]	valid_0's l2: 4.68299
[62]	valid_0's l2: 4.66877
[63]	valid_0's l2: 4.65538
[64]	valid_0's l2: 4.64355
[65]	valid_0's l2: 4.63307
[66]	valid_0's l2: 4.62161
[67]	valid_0's l2: 4.60766
[68]	valid_0's l2: 4.59817
[69]	valid_0's l2: 4.58988
[70]	valid_0's l2: 4.58263
[71]	valid_0's l2: 4.57486
[72]	valid_0's l2: 4.56441
[73]	valid_0's l2: 4.55667
[74]	valid_0's l2: 4.54967
[75]	valid_0's l2: 4.54409
[76]	valid_0's l2: 4.53691
[77]	valid_0's l2: 4.52571
[78]	valid_0's l2: 4.51707
[79]	valid_0's l2: 4.50949
[80]	valid_0's l2: 4.49875
[81]	valid_0's l2: 4.49092
[82]	valid_0's l2: 4.48374
[83]	valid_0's l2: 4.47866
[84]	valid_0's l2: 4.46934
[85]	valid_0's l2: 4.46334
[86]	valid_0's l2: 4.45332
[87]	valid_0's l2: 4.44627
[88]	valid_0's l2: 4.43738
[89]	valid_0's l2: 4.4324
[90]	valid_0's l2: 4.42348
[91]	valid_0's l2: 4.41889
[92]	valid_0's l2: 4.41309
[9

[352]	valid_0's l2: 3.9707
[353]	valid_0's l2: 3.97047
[354]	valid_0's l2: 3.96986
[355]	valid_0's l2: 3.96948
[356]	valid_0's l2: 3.96909
[357]	valid_0's l2: 3.96893
[358]	valid_0's l2: 3.9683
[359]	valid_0's l2: 3.96774
[360]	valid_0's l2: 3.96725
[361]	valid_0's l2: 3.96687
[362]	valid_0's l2: 3.96648
[363]	valid_0's l2: 3.96606
[364]	valid_0's l2: 3.9659
[365]	valid_0's l2: 3.96565
[366]	valid_0's l2: 3.96542
[367]	valid_0's l2: 3.96489
[368]	valid_0's l2: 3.96471
[369]	valid_0's l2: 3.96456
[370]	valid_0's l2: 3.96437
[371]	valid_0's l2: 3.96398
[372]	valid_0's l2: 3.96364
[373]	valid_0's l2: 3.96362
[374]	valid_0's l2: 3.9634
[375]	valid_0's l2: 3.96317
[376]	valid_0's l2: 3.96285
[377]	valid_0's l2: 3.96242
[378]	valid_0's l2: 3.96183
[379]	valid_0's l2: 3.96149
[380]	valid_0's l2: 3.96123
[381]	valid_0's l2: 3.96073
[382]	valid_0's l2: 3.9608
[383]	valid_0's l2: 3.96073
[384]	valid_0's l2: 3.96059
[385]	valid_0's l2: 3.96032
[386]	valid_0's l2: 3.96012
[387]	valid_0's l2: 3.959

...

[293]	valid_0's l2: 4.02437
[294]	valid_0's l2: 4.0241
[295]	valid_0's l2: 4.0237
[296]	valid_0's l2: 4.02285
[297]	valid_0's l2: 4.02173
[298]	valid_0's l2: 4.02118
[299]	valid_0's l2: 4.02052
[300]	valid_0's l2: 4.01908
[301]	valid_0's l2: 4.01863
[302]	valid_0's l2: 4.01726
[303]	valid_0's l2: 4.01661
[304]	valid_0's l2: 4.0153
[305]	valid_0's l2: 4.0146
[306]	valid_0's l2: 4.01322
[307]	valid_0's l2: 4.01168
[308]	valid_0's l2: 4.01063
[309]	valid_0's l2: 4.01019
[310]	valid_0's l2: 4.00939
[311]	valid_0's l2: 4.00849
[312]	valid_0's l2: 4.00813
[313]	valid_0's l2: 4.00754
[314]	valid_0's l2: 4.00715
[315]	valid_0's l2: 4.00685
[316]	valid_0's l2: 4.00609
[317]	valid_0's l2: 4.00569
[318]	valid_0's l2: 4.00493
[319]	valid_0's l2: 4.00419
[320]	valid_0's l2: 4.00352
[321]	valid_0's l2: 4.0029
[322]	valid_0's l2: 4.00241
[323]	valid_0's l2: 4.00185
[324]	valid_0's l2: 4.00098
[325]	valid_0's l2: 4.00047
[326]	valid_0's l2: 3.99976
[327]	valid_0's l2: 3.9988
[328]	valid_0's l2: 3.9985

[587]	valid_0's l2: 3.92383
[588]	valid_0's l2: 3.92374
[589]	valid_0's l2: 3.92353
[590]	valid_0's l2: 3.923
[591]	valid_0's l2: 3.92282
[592]	valid_0's l2: 3.92265
[593]	valid_0's l2: 3.92251
[594]	valid_0's l2: 3.92207
[595]	valid_0's l2: 3.92195
[596]	valid_0's l2: 3.92157
[597]	valid_0's l2: 3.92121
[598]	valid_0's l2: 3.92091
[599]	valid_0's l2: 3.92085
[600]	valid_0's l2: 3.9208
[601]	valid_0's l2: 3.92058
[602]	valid_0's l2: 3.92051
[603]	valid_0's l2: 3.92021
[604]	valid_0's l2: 3.92011
[605]	valid_0's l2: 3.91996
[606]	valid_0's l2: 3.91978
[607]	valid_0's l2: 3.91976
[608]	valid_0's l2: 3.91946
[609]	valid_0's l2: 3.91907
[610]	valid_0's l2: 3.91882
[611]	valid_0's l2: 3.91867
[612]	valid_0's l2: 3.91851
[613]	valid_0's l2: 3.91837
[614]	valid_0's l2: 3.91817
[615]	valid_0's l2: 3.91803
[616]	valid_0's l2: 3.91791
[617]	valid_0's l2: 3.91771
[618]	valid_0's l2: 3.9175
[619]	valid_0's l2: 3.91746
[620]	valid_0's l2: 3.91697
[621]	valid_0's l2: 3.91679
[622]	valid_0's l2: 3.91

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Successfully submitted to M5 Forecasting - Accuracy



  0%|          | 0.00/20.5M [00:00<?, ?B/s]
  0%|          | 8.00k/20.5M [00:00<09:41, 37.1kB/s]
  0%|          | 96.0k/20.5M [00:00<06:53, 51.8kB/s]
  2%|1         | 360k/20.5M [00:00<04:49, 73.3kB/s] 
  7%|6         | 1.42M/20.5M [00:00<03:12, 104kB/s]
 11%|#         | 2.26M/20.5M [00:00<02:09, 148kB/s]
 13%|#3        | 2.73M/20.5M [00:02<01:46, 175kB/s]
 31%|###1      | 6.46M/20.5M [00:02<00:59, 249kB/s]
 38%|###7      | 7.80M/20.5M [00:02<00:37, 353kB/s]
 44%|####4     | 9.08M/20.5M [00:02<00:24, 498kB/s]
 50%|#####     | 10.3M/20.5M [00:02<00:15, 698kB/s]
 56%|#####5    | 11.5M/20.5M [00:02<00:09, 973kB/s]
 62%|######1   | 12.7M/20.5M [00:02<00:06, 1.34MB/s]
 67%|######7   | 13.8M/20.5M [00:03<00:03, 1.83MB/s]
 73%|#######2  | 15.0M/20.5M [00:03<00:02, 2.45MB/s]
 78%|#######8  | 16.1M/20.5M [00:03<00:01, 3.21MB/s]
 84%|########3 | 17.3M/20.5M [00:03<00:00, 4.11MB/s]
 90%|########9 | 18.4M/20.5M [00:03<00:00, 5.11MB/s]
 95%|#########5| 19.5M/20.5M [00:03<00:00, 6.15MB/s]
100%|####

In [34]:
write_params_features(features, params, eval_results, mean_score)
save_feature_importance(feature_importances)

C:\Users\KJB\data_analysis\10_estimate_sales_of_walmart_retail_goods\mypackage\utils.py:134: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  fi = pd.concat([fi, tmp])
C:\Users\KJB\data_analysis\10_estimate_sales_of_walmart_retail_goods\mypackage\utils.py:135: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  fi = pd.concat([fi, feature])
